# ✅ Proyecto Final

### Análisis de Datos de Emprendedores en la Provincia de Entre Ríos (2016-2019)

### CoderHouse: Data Science III - Data Science III: NLP & Deep Learning aplicado a Ciencia de Datos

### Rodrigo Sosa - Cohorte 2024/2025 
**LinkedIn**: [https://www.linkedin.com/in/sosarodrigo/](https://www.linkedin.com/in/sosarodrigo/)

![Portada](https://github.com/sosarodrigox/coderhouse_emprendedores_it/blob/main/DataBanner2.png?raw=1)

DataBanner2.pngDataBanner2.png

📚 **Introducción**

El dataset utilizado en este análisis proviene de un archivo de Google Sheets asociado al formulario de Google desarrollado para la **Línea de Acción "Incorporación de Tecnología"**, un programa implementado por el Ministerio de Desarrollo Social de Entre Ríos entre marzo de 2016 y agosto de 2019. Este formulario recopiló información clave de emprendedores interesados en recibir apoyo financiero para la adquisición de herramientas y equipamiento.

✨ **Proyectos Previos**
En la etapa Data Science II de la Carrera de Ciencia de Datos, realicé dos proyectos relacionados:

1. **EDA y Data Wrangling**:
   - Objetivo: Explorar y limpiar los datos para identificar patrones relacionados con las aprobaciones.
   - Principales hallazgos: Factores como dedicación semanal, sector productivo y participación en programas de apoyo estaban fuertemente correlacionados con las aprobaciones.

2. **Machine Learning**:
   - Objetivo: Desarrollar un modelo predictivo para predecir la aprobación de proyectos.
   - Principal desafío: Un desbalance significativo en las clases (1,174 aprobados vs. 75 no aprobados), lo que dificultó la correcta identificación de los casos no aprobados.
   - Resultados: Aunque se aplicaron técnicas como SMOTE y modelos avanzados como XGBoost, el desbalanceo de datos limitó la capacidad predictiva del modelo para identificar proyectos no aprobados.

![Distribución de aprobados y no aprobados](img/aprobados.png)

★ **Enfoque del Proyecto Actual**

En este proyecto, se propone explorar un enfoque basado en **Procesamiento de Lenguaje Natural (NLP)**, utilizando las observaciones y conclusiones registradas por el equipo técnico en la columna *"Evaluación Final"*. Este texto contiene descripciones detalladas que podrían influir en el resultado de aprobación o desaprobación de los emprendedores.

✨ **Objetivo Principal**
Desarrollar un modelo predictivo que clasifique los proyectos como "aprobados" o "no aprobados" a partir del análisis de texto en las evaluaciones. Esto se logrará mediante técnicas de NLP para procesar y vectorizar el texto, combinado con algoritmos de clasificación supervisada.

✨ **Estrategia Metodológica**

1. **Preprocesamiento del Texto:**
   - Tokenización, lematización y remoción de *stopwords*.
   - Análisis de frecuencia y creación de representaciones vectoriales (TF-IDF).

2. **Análisis de Sentimientos:**
   - Evaluar la polaridad y el sentimiento general del texto como una característica adicional.

3. **Modelo Predictivo:**
   - Entrenar un modelo supervisado basado en las representaciones vectoriales del texto y evaluar su desempeño con métricas clave como Recall, F1-score y AUC-ROC.

✨ **Relevancia**
Este análisis tiene como objetivo aportar una nueva perspectiva al proceso de selección de proyectos, integrando herramientas de NLP para comprender mejor el impacto del lenguaje utilizado en las evaluaciones técnicas. Además, busca superar las limitaciones de los modelos de Machine Learning tradicionales al incorporar información contextual valiosa directamente desde los datos textuales.

Como **Técnico en Gestión y Administración Pública (UNER)** y **Técnico en Ciencia de Datos (ISPC)**, mi interés radica en aplicar técnicas avanzadas de ciencia de datos para mejorar la eficiencia de las políticas públicas. Este proyecto representa una oportunidad para explorar nuevos enfoques en la toma de decisiones, optimizando los recursos disponibles y apoyando el desarrollo de los emprendedores locales. 📊